# House pricing kaggle competition

Clean code dosent need explenation :^)

In [2]:
#Import is import-ant
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import cm
from sklearn.model_selection import train_test_split
import kaggle.api as kg
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_log_error
from os import path

In [6]:
if not(path.isfile("data/train.csv")):
    kg.competition_download_files(competition='house-prices-advanced-regression-techniques', path='data/')

In [7]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [8]:
train.head(5)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [9]:
train.shape

(1460, 81)

In [10]:
train.SalePrice.describe()

count      1460.000000
mean     180921.195890
std       79442.502883
min       34900.000000
25%      129975.000000
50%      163000.000000
75%      214000.000000
max      755000.000000
Name: SalePrice, dtype: float64

In [11]:
train.Street.describe()

count     1460
unique       2
top       Pave
freq      1454
Name: Street, dtype: object

In [22]:
train.loc[train.Street != 'Pave']['Street']

52      Grvl
335     Grvl
582     Grvl
812     Grvl
1061    Grvl
1184    Grvl
Name: Street, dtype: object

In [23]:
train.Street.unique()

array(['Pave', 'Grvl'], dtype=object)

In [24]:
train.Street.value_counts()

Pave    1454
Grvl       6
Name: Street, dtype: int64

In [25]:
train.Street = train.Street.map(lambda x: 0 if(x == 'Pave')else 1)

In [27]:
train.Street.describe()

count    1460.000000
mean        0.004110
std         0.063996
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         1.000000
Name: Street, dtype: float64

In [40]:
for col in train:
    if(train[col].dtype == object):
        print(train[col].unique())
        print(len(train[col].unique()))
        train = train.drop([col], axis=1) 

['RL' 'RM' 'C (all)' 'FV' 'RH']
5
[nan 'Grvl' 'Pave']
3
['Reg' 'IR1' 'IR2' 'IR3']
4
['Lvl' 'Bnk' 'Low' 'HLS']
4
['AllPub' 'NoSeWa']
2
['Inside' 'FR2' 'Corner' 'CulDSac' 'FR3']
5
['Gtl' 'Mod' 'Sev']
3
['CollgCr' 'Veenker' 'Crawfor' 'NoRidge' 'Mitchel' 'Somerst' 'NWAmes'
 'OldTown' 'BrkSide' 'Sawyer' 'NridgHt' 'NAmes' 'SawyerW' 'IDOTRR'
 'MeadowV' 'Edwards' 'Timber' 'Gilbert' 'StoneBr' 'ClearCr' 'NPkVill'
 'Blmngtn' 'BrDale' 'SWISU' 'Blueste']
25
['Norm' 'Feedr' 'PosN' 'Artery' 'RRAe' 'RRNn' 'RRAn' 'PosA' 'RRNe']
9
['Norm' 'Artery' 'RRNn' 'Feedr' 'PosN' 'PosA' 'RRAn' 'RRAe']
8
['1Fam' '2fmCon' 'Duplex' 'TwnhsE' 'Twnhs']
5
['2Story' '1Story' '1.5Fin' '1.5Unf' 'SFoyer' 'SLvl' '2.5Unf' '2.5Fin']
8
['Gable' 'Hip' 'Gambrel' 'Mansard' 'Flat' 'Shed']
6
['CompShg' 'WdShngl' 'Metal' 'WdShake' 'Membran' 'Tar&Grv' 'Roll'
 'ClyTile']
8
['VinylSd' 'MetalSd' 'Wd Sdng' 'HdBoard' 'BrkFace' 'WdShing' 'CemntBd'
 'Plywood' 'AsbShng' 'Stucco' 'BrkComm' 'AsphShn' 'Stone' 'ImStucc'
 'CBlock']
15
['VinylSd' 'M

In [41]:
train.shape

(1460, 39)

In [52]:
X = train.drop(['SalePrice'], axis=1)
y = train.SalePrice

In [53]:
X_trainPrep, X_testFinal, y_trainPrep, y_testFinal = train_test_split(X, y, test_size=0.33, random_state=42)

In [54]:
X_train, X_test, y_train, y_test = train_test_split(X_trainPrep, y_trainPrep, test_size=0.33, random_state=42)

In [58]:
import xgboost as xgb
from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error

In [59]:
xgb_model = xgb.XGBRegressor(objective="reg:linear", random_state=42)

xgb_model.fit(X_train, y_train)

y_pred = xgb_model.predict(X_test)

mse=mean_squared_error(y_test, y_pred)

print(np.sqrt(mse))

[22:23:39] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
26868.72720562952


In [60]:
np.sqrt(mean_squared_log_error( y_test, y_pred ))


0.13903801012930844